# Obtain Data from NSIDC DAAC - spatially/temporally subsetted to Bering Sea



## Import packages


In [1]:
import requests
import getpass
import socket
import json
import zipfile
import io
import math
import os
import shutil
import pprint
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona
import h5py
import re
# To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
from shapely.geometry import Polygon, mapping
from shapely.geometry.polygon import orient
from statistics import mean
from requests.auth import HTTPBasicAuth

In [2]:
# Earthdata Login credentials

# Enter your Earthdata Login user name
uid = 'chorvat'
# Enter your email address associated with your Earthdata Login account
email = 'christopher_horvat@brown.edu'
pswd = getpass.getpass('Earthdata Login password: ')

Earthdata Login password:  ·········


In [3]:
# Request token from Common Metadata Repository using Earthdata credentials
token_api_url = 'https://cmr.earthdata.nasa.gov/legacy-services/rest/tokens'
hostname = socket.gethostname()
ip = socket.gethostbyname(hostname)

data = {
    'token': {
        'username': uid,
        'password': pswd,
        'client_id': 'NSIDC_client_id',
        'user_ip_address': ip
    }
}
headers={'Accept': 'application/json'}
response = requests.post(token_api_url, json=data, headers=headers)
token = json.loads(response.content)['token']['id']
print(token)

55426424-09AF-2D81-02AE-85ECA842F7C4


In [4]:
# Input data set ID (e.g. ATL07) of interest here, also known as "short name".
# We're gonna do this with the sea ice product. 
short_name = 'ATL07'
latest_version = '001'


In [5]:
# Input temporal range 

# Input start date in yyyy-MM-dd format
start_date = '2018-12-01'
# Input start time in HH:mm:ss format
start_time = '00:00:00'
# Input end date in yyyy-MM-dd format
end_date = '2018-12-06'
# Input end time in HH:mm:ss format
end_time = '23:59:59'

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'
print(temporal)

2018-12-01T00:00:00Z,2018-12-06T23:59:59Z


In [6]:
# # Commenting for tutorial since we will be walking through option 3 (spatial file input) together
# # Bounding Box spatial parameter in 'W,S,E,N' format

# Input bounding box
# Input lower left longitude in decimal degrees
LL_lon = '-180'
# Input lower left latitude in decimal degrees
LL_lat = '60'
# Input upper right longitude in decimal degrees
UR_lon = '-160'
# Input upper right latitude in decimal degrees
UR_lat = '85'
# aoi value used for CMR params below

bounding_box = LL_lon + ',' + LL_lat + ',' + UR_lon + ',' + UR_lat
bbox = bounding_box
# aoi value used for CMR params below
aoi = '1'
print(bounding_box)


-180,60,-160,85


In [7]:
params = {
    'short_name': short_name,
    'version': latest_version,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1,
    'bounding_box': bounding_box
    }
    
# Query number of granules using our (paging over results)

granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'

granules = []
while True:
    response = requests.get(granule_search_url, params=params, headers=headers)
    results = json.loads(response.content)

    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    params['page_num'] += 1

    
# Get number of granules over my area and time of interest
len(granules)

31

In [8]:
# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

print(capability_url)

https://n5eil02u.ecs.nsidc.org/egi/capabilities/ATL07.001.xml


All of NSIDC's service endpoints are behind NASA Earthdata Login. We need to create a session to store cookies and pass Earthdata Login credentials to capabilities url.

In [9]:
# Create session to store cookie and pass credentials to capabilities url

session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))

root = ET.fromstring(response.content)

From the service capability XML, we can collect lists with each service option to gather service information.

In [10]:
# collect lists with each service option

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

# variable subsetting
variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
variables_raw = [variables[i]['value'] for i in range(len(variables))]
variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
variable_vals = [v.replace(':', '/') for v in variables_join]

# reformatting
formats = [Format.attrib for Format in root.iter('Format')]
format_vals = [formats[i]['value'] for i in range(len(formats))]
format_vals.remove('')

# reprojection only applicable on ICESat-2 L3B products, yet to be available. 

# reformatting options that support reprojection
normalproj = [Projections.attrib for Projections in root.iter('Projections')]
normalproj_vals = []
normalproj_vals.append(normalproj[0]['normalProj'])
format_proj = normalproj_vals[0].split(',')
format_proj.remove('')
format_proj.append('No reformatting')

#reprojection options
projections = [Projection.attrib for Projection in root.iter('Projection')]
proj_vals = []
for i in range(len(projections)):
    if (projections[i]['value']) != 'NO_CHANGE' :
        proj_vals.append(projections[i]['value'])
        
# reformatting options that do not support reprojection
no_proj = [i for i in format_vals if i not in format_proj]

#### Let's confirm that subset services exist for our data set by reviewing the `subagent` list. If the list contains service information, we know that services are available. If not, we need to set the `agent` API parameter to `NO` to indicate that our request will bypass the subsetter. This will quickly send back the data "natively" without any customization applied.

In [11]:
print(subagent)
if len(subagent) < 1 :
    agent = 'NO'

[{'id': 'ICESAT2', 'spatialSubsetting': 'true', 'spatialSubsettingShapefile': 'true', 'temporalSubsetting': 'true', 'type': 'both', 'maxGransSyncRequest': '100', 'maxGransAsyncRequest': '2000'}]


More information is contained in the subagent list, including the maximum number of granules that we can request per order depending on our configuration. We'll come back to these options below.

#### Let's start with spatial subsetting. Recall that there are three options to *filter* our search results by spatial constraint: 

1) Bounding Box: Corresponding to the CMR `bounding_box` KVP

2) Polygon coordinate pairs: Corresponding to the CMR `polygon` KVP

3) Spatial file input, including Esri Shapefile or KML/KMZ: We simplified the file input to also be read by the CMR `polygon` KVP 
    
#### We see above that `spatialSubsetting` is `true` and `spatialSubsettingShapefile` is `true`. Therefore the same *filtering* options can be applied to our *subset* constraint, with unique KVPs for the subsetting service:

1) Bounding Box: `bbox` subset KVP

2) Polygon coordinate pairs: `bounding_shape` subset KVP in [GeoJSON](https://geojson.org/) format. 

3) Spatial file input: The file can be read directly by the subsetter without simplification. This file will be posted to the API endpoint, so we don't need to specify an additional subset KVP here. 

#### Because we're pursuing option 3), we don't need to provide an additional subset parameter. Below is commented code for bounding box inputs.

In [12]:
# Temporal subsetting KVP

timevar = start_date + 'T' + start_time + ',' + end_date + 'T' + end_time
print(timevar)

# print(format_vals)
# pprint.pprint(variable_vals)

2018-12-01T00:00:00,2018-12-06T23:59:59


#### Next, let's explore the reformatting options available.


We can view the entire list of variables if desired:

In [13]:
coverage = '/gt1l/sea_ice_segments/delta_time,\
/gt1l/sea_ice_segments/latitude,\
/gt1l/sea_ice_segments/longitude,\
/gt1l/sea_ice_segments/seg_dist_x,\
/gt1l/sea_ice_segments/heights/across_track_distance,\
/gt1l/sea_ice_segments/heights/height_segment_height,\
/gt1l/sea_ice_segments/heights/height_segment_fit_quality_flag,\
/gt1l/sea_ice_segments/heights/height_segment_confidence,\
/gt1l/sea_ice_segments/heights/height_segment_length_seg,\
/gt1l/sea_ice_segments/heights/height_segment_type,\
/gt1l/sea_ice_segments/heights/height_segment_w_gaussian,\
/gt1l/sea_ice_segments/heights/height_segment_rms,\
/gt1l/sea_ice_segments/heights/height_segment_ssh_flag,\
/gt1l/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt1l/sea_ice_segments/geophysical/height_segment_dac,\
/gt1l/sea_ice_segments/geophysical/height_segment_mss,\
/gt1l/sea_ice_segments/geophysical/height_segment_geoid,\
/gt1l/ancillary_data/atlas_sdp_gps_epoch,\
/gt1l/sea_ice_segments/stats/ice_conc,\
/gt1r/sea_ice_segments/delta_time,\
/gt1r/sea_ice_segments/latitude,\
/gt1r/sea_ice_segments/longitude,\
/gt1r/sea_ice_segments/seg_dist_x,\
/gt1r/sea_ice_segments/heights/across_track_distance,\
/gt1r/sea_ice_segments/heights/height_segment_height,\
/gt1r/sea_ice_segments/heights/height_segment_fit_quality_flag,\
/gt1r/sea_ice_segments/heights/height_segment_confidence,\
/gt1r/sea_ice_segments/heights/height_segment_length_seg,\
/gt1r/sea_ice_segments/heights/height_segment_type,\
/gt1r/sea_ice_segments/heights/height_segment_w_gaussian,\
/gt1r/sea_ice_segments/heights/height_segment_rms,\
/gt1r/sea_ice_segments/heights/height_segment_ssh_flag,\
/gt1r/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt1r/sea_ice_segments/geophysical/height_segment_dac,\
/gt1r/sea_ice_segments/geophysical/height_segment_mss,\
/gt1r/sea_ice_segments/geophysical/height_segment_geoid,\
/gt1r/ancillary_data/atlas_sdp_gps_epoch,\
/gt1r/sea_ice_segments/stats/ice_conc,\
/gt2l/sea_ice_segments/delta_time,\
/gt2l/sea_ice_segments/latitude,\
/gt2l/sea_ice_segments/longitude,\
/gt2l/sea_ice_segments/seg_dist_x,\
/gt2l/sea_ice_segments/heights/across_track_distance,\
/gt2l/sea_ice_segments/heights/height_segment_height,\
/gt2l/sea_ice_segments/heights/height_segment_fit_quality_flag,\
/gt2l/sea_ice_segments/heights/height_segment_confidence,\
/gt2l/sea_ice_segments/heights/height_segment_length_seg,\
/gt2l/sea_ice_segments/heights/height_segment_type,\
/gt2l/sea_ice_segments/heights/height_segment_w_gaussian,\
/gt2l/sea_ice_segments/heights/height_segment_rms,\
/gt2l/sea_ice_segments/heights/height_segment_ssh_flag,\
/gt2l/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt2l/sea_ice_segments/geophysical/height_segment_dac,\
/gt2l/sea_ice_segments/geophysical/height_segment_mss,\
/gt2l/sea_ice_segments/geophysical/height_segment_geoid,\
/gt2l/ancillary_data/atlas_sdp_gps_epoch,\
/gt2l/sea_ice_segments/stats/ice_conc,\
/gt2r/sea_ice_segments/delta_time,\
/gt2r/sea_ice_segments/latitude,\
/gt2r/sea_ice_segments/longitude,\
/gt2r/sea_ice_segments/seg_dist_x,\
/gt2r/sea_ice_segments/heights/across_track_distance,\
/gt2r/sea_ice_segments/heights/height_segment_height,\
/gt2r/sea_ice_segments/heights/height_segment_fit_quality_flag,\
/gt2r/sea_ice_segments/heights/height_segment_confidence,\
/gt2r/sea_ice_segments/heights/height_segment_length_seg,\
/gt2r/sea_ice_segments/heights/height_segment_type,\
/gt2r/sea_ice_segments/heights/height_segment_w_gaussian,\
/gt2r/sea_ice_segments/heights/height_segment_rms,\
/gt2r/sea_ice_segments/heights/height_segment_ssh_flag,\
/gt2r/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt2r/sea_ice_segments/geophysical/height_segment_dac,\
/gt2r/sea_ice_segments/geophysical/height_segment_mss,\
/gt2r/sea_ice_segments/geophysical/height_segment_geoid,\
/gt2r/ancillary_data/atlas_sdp_gps_epoch,\
/gt2r/sea_ice_segments/stats/ice_conc,\
/gt3l/sea_ice_segments/delta_time,\
/gt3l/sea_ice_segments/latitude,\
/gt3l/sea_ice_segments/longitude,\
/gt3l/sea_ice_segments/seg_dist_x,\
/gt3l/sea_ice_segments/heights/across_track_distance,\
/gt3l/sea_ice_segments/heights/height_segment_height,\
/gt3l/sea_ice_segments/heights/height_segment_fit_quality_flag,\
/gt3l/sea_ice_segments/heights/height_segment_confidence,\
/gt3l/sea_ice_segments/heights/height_segment_length_seg,\
/gt3l/sea_ice_segments/heights/height_segment_type,\
/gt3l/sea_ice_segments/heights/height_segment_w_gaussian,\
/gt3l/sea_ice_segments/heights/height_segment_rms,\
/gt3l/sea_ice_segments/heights/height_segment_ssh_flag,\
/gt3l/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt3l/sea_ice_segments/geophysical/height_segment_dac,\
/gt3l/sea_ice_segments/geophysical/height_segment_mss,\
/gt3l/sea_ice_segments/geophysical/height_segment_geoid,\
/gt3l/ancillary_data/atlas_sdp_gps_epoch,\
/gt3l/sea_ice_segments/stats/ice_conc,\
/gt3r/sea_ice_segments/delta_time,\
/gt3r/sea_ice_segments/latitude,\
/gt3r/sea_ice_segments/longitude,\
/gt3r/sea_ice_segments/seg_dist_x,\
/gt3r/sea_ice_segments/heights/across_track_distance,\
/gt3r/sea_ice_segments/heights/height_segment_height,\
/gt3r/sea_ice_segments/heights/height_segment_fit_quality_flag,\
/gt3r/sea_ice_segments/heights/height_segment_confidence,\
/gt3r/sea_ice_segments/heights/height_segment_length_seg,\
/gt3r/sea_ice_segments/heights/height_segment_type,\
/gt3r/sea_ice_segments/heights/height_segment_w_gaussian,\
/gt3r/sea_ice_segments/heights/height_segment_rms,\
/gt3r/sea_ice_segments/heights/height_segment_ssh_flag,\
/gt3r/sea_ice_segments/heights/height_segment_surface_error_est,\
/gt3r/sea_ice_segments/geophysical/height_segment_dac,\
/gt3r/sea_ice_segments/geophysical/height_segment_mss,\
/gt3r/sea_ice_segments/geophysical/height_segment_geoid,\
/gt3r/ancillary_data/atlas_sdp_gps_epoch,\
/gt3r/sea_ice_segments/stats/ice_conc'

In [14]:
#Set NSIDC data access base URL
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'

In [15]:
# Set number of granules requested per order, which we will initially set to 10.
page_size = 20

#Determine number of pages basd on page_size and total granules. Loop requests by this value
page_num = math.ceil(len(granules)/page_size)

#Set request mode. 
request_mode = 'async'

# Determine how many individual orders we will request based on the number of granules requested

#### After all of these KVP inputs, what does our request look like? Here's a summary of all possible KVPs that we explored, both for CMR searching and for the subsetter:

#### CMR search keys:
* `short_name=`
* `version=`
* `temporal=`
* `bounding_box=`
* `polygon=`

#### Customization service keys:
* `time=`
* `bbox=`
* `bounding_shape=` 
* `format=`
* `projection=`
* `projection_parameters=`
* `Coverage=`

#### No customization (access only):
* `agent=`    
* `include_meta=` 
    * `Y` by default. `N` for No metadata requested.

#### Request configuration keys:
* `request_mode=` 
* `page_size=`
* `page_num=`
* `token=`
* `email=`

#### If we were to create an API request based on our request parameters and submit into a web browser for example, here's what we end up with:

In [16]:
print(bbox)
subset_params = {
    'short_name': short_name, 
#    'version': latest_version, 
    'temporal': temporal, 
#    'time': timevar, 
    'bounding_box': bbox, 
    'Coverage': coverage, 
    'request_mode': request_mode, 
    'page_size': page_size,  
    'token': token, 
    'email': email, 
    }
print(subset_params)

-180,60,-160,85
{'short_name': 'ATL07', 'temporal': '2018-12-01T00:00:00Z,2018-12-06T23:59:59Z', 'bounding_box': '-180,60,-160,85', 'Coverage': '/gt1l/sea_ice_segments/delta_time,/gt1l/sea_ice_segments/latitude,/gt1l/sea_ice_segments/longitude,/gt1l/sea_ice_segments/seg_dist_x,/gt1l/sea_ice_segments/heights/across_track_distance,/gt1l/sea_ice_segments/heights/height_segment_height,/gt1l/sea_ice_segments/heights/height_segment_fit_quality_flag,/gt1l/sea_ice_segments/heights/height_segment_confidence,/gt1l/sea_ice_segments/heights/height_segment_length_seg,/gt1l/sea_ice_segments/heights/height_segment_type,/gt1l/sea_ice_segments/heights/height_segment_w_gaussian,/gt1l/sea_ice_segments/heights/height_segment_rms,/gt1l/sea_ice_segments/heights/height_segment_ssh_flag,/gt1l/sea_ice_segments/heights/height_segment_surface_error_est,/gt1l/sea_ice_segments/geophysical/height_segment_dac,/gt1l/sea_ice_segments/geophysical/height_segment_mss,/gt1l/sea_ice_segments/geophysical/height_segment_geoi

#### We'll request the same data but without any subsetting services applied. Let's create another request parameter dictionary with the `time` and `coverage` service keys removed, and we'll add `agent=NO` instead.

## Request Data

#### Finally, we'll download the data directly to this notebook directory in a new Outputs folder. The progress of each order will be reported.

We'll start by creating an output folder if the folder does not already exist.

In [17]:
path = '/home/jovyan/Data/'
if not os.path.exists(path):
    os.mkdir(path)

First we'll submit our request without subsetting services:

In [18]:
# Request data service for each page number, and unzip outputs

for i in range(page_num):
    page_val = i + 1
    print('Order: ', page_val)
    subset_params.update( {'page_num': page_val} )
    
# Post polygon to API endpoint for polygon subsetting to subset based on original, non-simplified KML file

#    shape_post = {'shapefile': open(kml_filepath, 'rb')}
#    request = session.post(base_url, params=subset_params, files=shape_post) 
    
# FOR ALL OTHER REQUESTS THAT DO NOT UTILIZED AN UPLOADED POLYGON FILE, USE A GET REQUEST INSTEAD OF POST:
    request = session.get(base_url, params=subset_params)
    
    print('Request HTTP response: ', request.status_code)

# Raise bad request: Loop will stop for bad response code.
    request.raise_for_status()
    print('Order request URL: ', request.url)
    esir_root = ET.fromstring(request.content)
    print('Order request response XML content: ', request.content)

# Look up order ID
    orderlist = []   
    for order in esir_root.findall("./order/"):
        orderlist.append(order.text)
    orderID = orderlist[0]
    print('order ID: ', orderID)

# Create status URL
    statusURL = base_url + '/' + orderID
    print('status URL: ', statusURL)

# Find order status
    request_response = session.get(statusURL)    
    print('HTTP response from order response URL: ', request_response.status_code)
    
# Raise bad request: Loop will stop for bad response code.
    request_response.raise_for_status()
    request_root = ET.fromstring(request_response.content)
    statuslist = []
    for status in request_root.findall("./requestStatus/"):
        statuslist.append(status.text)
    status = statuslist[0]
    print('Data request ', page_val, ' is submitting...')
    print('Initial request status is ', status)

# Continue to loop while request is still processing
    while status == 'pending' or status == 'processing': 
        print('Status is not complete. Trying again.')
        time.sleep(10)
        loop_response = session.get(statusURL)

# Raise bad request: Loop will stop for bad response code.
        loop_response.raise_for_status()
        loop_root = ET.fromstring(loop_response.content)

# Find status
        statuslist = []
        for status in loop_root.findall("./requestStatus/"):
            statuslist.append(status.text)
        status = statuslist[0]
        print('Retry request status is: ', status)
        if status == 'pending' or status == 'processing':
            continue

# Order can either complete, complete_with_errors, or fail:
# Provide complete_with_errors error message:
    if status == 'complete_with_errors' or status == 'failed':
        messagelist = []
        for message in loop_root.findall("./processInfo/"):
            messagelist.append(message.text)
        print('error messages:')
        pprint.pprint(messagelist)

# Download zipped order if status is complete or complete_with_errors
    if status == 'complete' or status == 'complete_with_errors':
        downloadURL = 'https://n5eil02u.ecs.nsidc.org/esir/' + orderID + '.zip'
        print('Zip download URL: ', downloadURL)
        print('Beginning download of zipped output...')
        zip_response = session.get(downloadURL)
        # Raise bad request: Loop will stop for bad response code.
        zip_response.raise_for_status()
        with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
            z.extractall(path)
        print('Data request', page_val, 'is complete.')
    else: print('Request failed.')

Order:  1
Request HTTP response:  201
Order request URL:  https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL07&temporal=2018-12-01T00%3A00%3A00Z%2C2018-12-06T23%3A59%3A59Z&bounding_box=-180%2C60%2C-160%2C85&Coverage=%2Fgt1l%2Fsea_ice_segments%2Fdelta_time%2C%2Fgt1l%2Fsea_ice_segments%2Flatitude%2C%2Fgt1l%2Fsea_ice_segments%2Flongitude%2C%2Fgt1l%2Fsea_ice_segments%2Fseg_dist_x%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Facross_track_distance%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Fheight_segment_height%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Fheight_segment_fit_quality_flag%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Fheight_segment_confidence%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Fheight_segment_length_seg%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Fheight_segment_type%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Fheight_segment_w_gaussian%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Fheight_segment_rms%2C%2Fgt1l%2Fsea_ice_segments%2Fheights%2Fheight_segment_ssh_flag%2C%2Fgt1l%2Fsea_ice_segments

In [19]:
#Clean up Outputs folder by removing individual granule folders 

print(path)

for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
        
for root, dirs, files in os.walk(path):
    for name in dirs:
        print(os.path.join(root, name))
        os.rmdir(os.path.join(root, name))

/home/jovyan/Data/
/home/jovyan/Data/161226428


OSError: [Errno 39] Directory not empty: '/home/jovyan/Data/161226428'

In [ ]:
#List files
sorted(os.listdir(path))